In [1]:

import pandas as pd 
import numpy as np

#stats
from scipy import sparse
import scipy.stats as ss

#viz
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud ,STOPWORDS
from PIL import Image

#nlp
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords

#import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

#FeatureEngineering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import textblob
#import xgboost

from textblob import TextBlob
#pip install textblob
import matplotlib.pyplot as plt
import os


In [4]:
#! pip install openpyxl
#from openpyxl import Workbook

     -------------------------------------- 242.1/242.1 kB 1.9 MB/s eta 0:00:00


In [2]:
df=pd.read_excel(r'C:/Users/Ankita/OneDrive - Infiniti Research/Desktop/Johnson&Johnson/Data/Review Anlaysis Data/Translated Mice Data.xlsx')

In [3]:
df.shape

(2437, 131)

In [4]:
df.head()

date        asin  year  weeknum  fiscal_year  fiscal_month  \
0  2020-12-20  B07ZPDV5NJ  2020       51         2020            12   
1  2020-12-20  B087ZG9VTV  2020       51         2020            12   
2  2020-12-21  B07BVW522M  2020       51         2020            12   
3  2020-12-21  B087ZFLY3K  2020       51         2020            12   
4  2020-12-21  B087ZFLY3K  2020       51         2020            12   

   first_party_ordered_sales_vc  first_party_ordered_sales_local_vc  \
0                          89.1                               78.02   
1                         304.0                              266.24   
2                         151.9                              133.03   
3                         105.0                               91.92   
4                         105.0                               91.92   

   first_party_ordered_units_vc  average_sales_price_vc  ...  \
0                            22                     3.5  ...   
1                            64                     4.2  ...   
2                            17                     7.8  ...   
3                            26                     3.5  ...   
4                            26                     3.5  ...   

   useful_ratings_negative  useful_ratings_positive  feature_wise_rating  \
0                        1                        3                    1   
1                        0                        1                    1   
2                        0                        4                    1   
3                        1                        3                    1   
4                        1                        3                    1   

   qna_answered                                              title  \
0             4  Neutrogena Anti-Pickel Gesichtspflege, SOS Sof...   
1             4  Neutrogena Curcuma Clear Beruhigender Reinigun...   
2             6  Neutrogena Hydro Boost Hyaluron Serum, Aqua Pe...   
3             4  Neutrogena Curcuma Clear Gesichtsreinigung, Be...   
4             4  Neutrogena Curcuma Clear Gesichtsreinigung, Be...   

                                             text_pr  \
0  Bei mir hilft es leider nicht. Beim Auftragen ...   
1  Wirkt und trocknet die Haut nicht aus. Habe sc...   
2                                                 Ja   
3  Brennt auf der Haut, danach ist die Haut schli...   
4                                                ☹☹☹   

                                  text_pr_translated  \
0  Unfortunately it doesn't help me. A burning se...   
1  Effective and does not dry out the skin. I've ...   
2                                                And   
3  Burns on the skin, then the skin is worse than...   
4                                                NaN   

                                    translated_title  text_sentiment_sc  \
0  Neutrogena anti-pimple face care, SOS immediat...           0.040000   
1  Neutrogena Curcuma Clear Soothing cleansing fo...           0.383333   
2  Neutrogena Hydro Boost Hyaluron Serum, aqua pe...           0.000000   
3  Neutrogena Curcuma Clear facial cleansing, soo...          -0.122222   
4  Neutrogena Curcuma Clear facial cleansing, soo...           0.000000   

   date&asin  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  

[5 rows x 131 columns]

In [8]:
df.columns

Index(['date', 'asin', 'year', 'weeknum', 'fiscal_year', 'fiscal_month',
       'first_party_ordered_sales_vc', 'first_party_ordered_sales_local_vc',
       'first_party_ordered_units_vc', 'average_sales_price_vc',
       ...
       'useful_ratings_negative', 'useful_ratings_positive',
       'feature_wise_rating', 'qna_answered', 'title', 'text_pr',
       'text_pr_translated', 'translated_title', 'text_sentiment_sc',
       'date&asin'],
      dtype='object', length=131)

In [5]:
df['text_pr_translated'].isnull().sum()

18

Dropping Missing Values

In [6]:
df.dropna(subset=['translated_title','text_pr_translated'],inplace=True)

In [7]:
df['text_pr_translated']

0       Unfortunately it doesn't help me. A burning se...
1       Effective and does not dry out the skin. I've ...
2                                                     And
3       Burns on the skin, then the skin is worse than...
5       Burns on the skin, then the skin is worse than...
                              ...                        
2432    I am very satisfied with this Neutrogena Curcu...
2433    I have very dry skin and therefore tested Neut...
2434    The set comes in an inconspicuous, simple and ...
2435               It smells very good and it feels fresh
2436    A good product...cleans well with a pleasant s...
Name: text_pr_translated, Length: 2419, dtype: object

In [8]:
df['text_pr_translated']= df['text_pr_translated'].astype(str)

In [9]:
df['text_pr_translated']

0       Unfortunately it doesn't help me. A burning se...
1       Effective and does not dry out the skin. I've ...
2                                                     And
3       Burns on the skin, then the skin is worse than...
5       Burns on the skin, then the skin is worse than...
                              ...                        
2432    I am very satisfied with this Neutrogena Curcu...
2433    I have very dry skin and therefore tested Neut...
2434    The set comes in an inconspicuous, simple and ...
2435               It smells very good and it feels fresh
2436    A good product...cleans well with a pleasant s...
Name: text_pr_translated, Length: 2419, dtype: object

# TEXT PREPROCESSING

In [10]:
stop = set(nltk.corpus.stopwords.words('english'))

#Abbrevations and Words correction
def clean_text(text):
    text = text.lower()
    text = text.strip()
    #text = text.replace('return factor',"return factor").replace('cons','')
    #text = re.sub(r' +', '', text)
    text = re.sub(r"[-()\"#/@;:{}`+=~|.!?,'0-9]","",text)
    return(text)

In [11]:
df['text_pr_translated'] = df['text_pr_translated'].apply(lambda x: clean_text(x))

In [12]:
df['text_pr_translated'] 

0       unfortunately it doesnt help me a burning sens...
1       effective and does not dry out the skin ive tr...
2                                                     and
3       burns on the skin then the skin is worse than ...
5       burns on the skin then the skin is worse than ...
                              ...                        
2432    i am very satisfied with this neutrogena curcu...
2433    i have very dry skin and therefore tested neut...
2434    the set comes in an inconspicuous simple and o...
2435               it smells very good and it feels fresh
2436    a good productcleans well with a pleasant smel...
Name: text_pr_translated, Length: 2419, dtype: object

In [13]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [14]:
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

#STOP WORD REMOVAL
df['text_pr_translated'] = df['text_pr_translated'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# STEMMING
#df['text_pr_translated'] = df['text_pr_translated'].apply(lambda x: " ".join(PorterStemmer().stem(word) for word in x.split()))

import nltk
nltk.stem.WordNetLemmatizer().lemmatize('word')


'word'

In [15]:
#LEMMITIZATION
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()

for words in df['text_pr_translated']:
    print(words + " ---> " + wnl.lemmatize(words))

unfortunately doesnt help burning sensation felt applying actually good thing works unfortunately pimples still come greatest splendor wont buy unfortunately ---> unfortunately doesnt help burning sensation felt applying actually good thing works unfortunately pimples still come greatest splendor wont buy unfortunately
effective dry skin ive tried many things im glad stumbled across product ---> effective dry skin ive tried many things im glad stumbled across product
 ---> 
burns skin skin worse feeling dry soothing something else sensitive skin keep hands ---> burns skin skin worse feeling dry soothing something else sensitive skin keep hands
burns skin skin worse feeling dry soothing something else sensitive skin keep hands ---> burns skin skin worse feeling dry soothing something else sensitive skin keep hands
since ive using curcuma clear skin gotten lot better still use moisturizer skin always feels tight washing im thrilled skin problems havent gone away theyve gotten lot better 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ankita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Splitting the dataframes into Acne Treatments and Moisturisers and Cleansers

In [29]:
df['date_asin']=df['date'] + df['asin']

In [42]:
df.set_index('date_asin',inplace=True)

In [43]:
df

date        asin  year  weeknum  fiscal_year  \
date_asin                                                                  
2020-12-20B07ZPDV5NJ  2020-12-20  B07ZPDV5NJ  2020       51         2020   
2020-12-20B087ZG9VTV  2020-12-20  B087ZG9VTV  2020       51         2020   
2020-12-21B07BVW522M  2020-12-21  B07BVW522M  2020       51         2020   
2020-12-21B087ZFLY3K  2020-12-21  B087ZFLY3K  2020       51         2020   
2020-12-21B087ZG94GD  2020-12-21  B087ZG94GD  2020       51         2020   
...                          ...         ...   ...      ...          ...   
2022-06-22B087ZG94GD  2022-06-22  B087ZG94GD  2022       25         2022   
2022-06-22B09GYRXP9P  2022-06-22  B09GYRXP9P  2022       25         2022   
2022-06-22B09NLL2YSF  2022-06-22  B09NLL2YSF  2022       25         2022   
2022-06-23B082VP1JR6  2022-06-23  B082VP1JR6  2022       25         2022   
2022-06-23B082VP1JR6  2022-06-23  B082VP1JR6  2022       25         2022   

                      fiscal_month  first_party_ordered_sales_vc  \
date_asin                                                          
2020-12-20B07ZPDV5NJ            12                          89.1   
2020-12-20B087ZG9VTV            12                         304.0   
2020-12-21B07BVW522M            12                         151.9   
2020-12-21B087ZFLY3K            12                         105.0   
2020-12-21B087ZG94GD            12                         289.1   
...                            ...                           ...   
2022-06-22B087ZG94GD             6                          56.8   
2022-06-22B09GYRXP9P             6                          60.1   
2022-06-22B09NLL2YSF             6                          84.5   
2022-06-23B082VP1JR6             6                         176.0   
2022-06-23B082VP1JR6             6                         176.0   

                      first_party_ordered_sales_local_vc  \
date_asin                                                  
2020-12-20B07ZPDV5NJ                               78.02   
2020-12-20B087ZG9VTV                              266.24   
2020-12-21B07BVW522M                              133.03   
2020-12-21B087ZFLY3K                               91.92   
2020-12-21B087ZG94GD                              253.21   
...                                                  ...   
2022-06-22B087ZG94GD                               47.38   
2022-06-22B09GYRXP9P                               50.10   
2022-06-22B09NLL2YSF                               70.50   
2022-06-23B082VP1JR6                              146.81   
2022-06-23B082VP1JR6                              146.81   

                      first_party_ordered_units_vc  average_sales_price_vc  \
date_asin                                                                    
2020-12-20B07ZPDV5NJ                            22                     3.5   
2020-12-20B087ZG9VTV                            64                     4.2   
2020-12-21B07BVW522M                            17                     7.8   
2020-12-21B087ZFLY3K                            26                     3.5   
2020-12-21B087ZG94GD                            71                     3.6   
...                                            ...                     ...   
2022-06-22B087ZG94GD                            11                     4.3   
2022-06-22B09GYRXP9P                             6                     8.3   
2022-06-22B09NLL2YSF                             3                    23.5   
2022-06-23B082VP1JR6                            41                     3.6   
2022-06-23B082VP1JR6                            41                     3.6   

                      ...  useful_ratings_negative  useful_ratings_positive  \
date_asin             ...                                                     
2020-12-20B07ZPDV5NJ  ...                        1                        3   
2020-12-20B087ZG9VTV  ...                        0                        1   
2020-12-21B07BVW522M  ...                 

In [44]:
df_acne = df[df.gsr_level_2 == 'Acne Treatments']
df_moisturiser = df[df.gsr_level_2 == 'Moisturisers']
df_cleansers = df[df.gsr_level_2 == 'Cleansers']

In [45]:
df_acne.shape, df_cleansers.shape,df_moisturiser.shape

((1302, 131), (418, 131), (699, 131))

In [46]:
df_cleansers.text_sentiment_sc.describe()

count    418.000000
mean       0.310610
std        0.287319
min       -0.700000
25%        0.115272
50%        0.302381
75%        0.500000
max        1.000000
Name: text_sentiment_sc, dtype: float64

# Splitting the dataframes into positive and negative sentiments

In [47]:
df_acne_pos=df_acne[df_acne.text_sentiment_sc>0]
df_acne_neg=df_acne[df_acne.text_sentiment_sc<0]

df_moisturiser_pos=df_moisturiser[df_moisturiser.text_sentiment_sc>0]
df_moisturiser_neg=df_moisturiser[df_moisturiser.text_sentiment_sc<0]

df_cleansers_pos=df_cleansers[df_cleansers.text_sentiment_sc>0]
df_cleansers_neg=df_cleansers[df_cleansers.text_sentiment_sc<0]

In [48]:
df_moisturiser_neg.text_pr_translated

date_asin
2021-01-19B087QX6CMP    cream arrived today disappointed upon unpackin...
2021-02-07B0849D9GB4          unfortunately nothing skin tense afterwards
2021-02-16B087QX6CMQ    product good tried sister cream arrived disgus...
2021-03-05B0849D9GB4    tolerate well moisturizes well dry patches fac...
2021-03-10B0849D9GB4    unfortunately cream didnt moisturize skin does...
                                              ...                        
2022-04-17B087QX6CMP       moisturizing however pack small wont last long
2022-04-26B087QX6CMQ    eyes ran put cream face googled unbelievable s...
2022-04-30B08TJVD6TW    cream moved starts go something face really ba...
2022-05-10B087QX6CMP    unfortunately dont tolerate cream similar seru...
2022-06-22B09GYRXP9P    dry skin therefore tested neutrogenas hyaluron...
Name: text_pr_translated, Length: 61, dtype: object

# Convert Text into Numerical Representation

In [49]:
# Converting text into numerical representation
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False)

# Converting text into numerical representation
cv_vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

In [50]:
# Array from TF-IDF Vectorizer 
tf_idf_arr = tf_idf_vectorizer.fit_transform(df_moisturiser_pos['text_pr_translated'])
tf_idf_arr1 = tf_idf_vectorizer.fit_transform(df_moisturiser_neg['text_pr_translated'])

# Array from Count Vectorizer 
cv_arr = cv_vectorizer.fit_transform(df['text_pr_translated'])
cv_arr1 = cv_vectorizer.fit_transform(df['text_pr_translated'])

In [51]:
# this is our converted text to numerical representation from the Tf-IDF vectorizer

tf_idf_arr

<586x51 sparse matrix of type '<class 'numpy.float64'>'
	with 12415 stored elements in Compressed Sparse Row format>

In [52]:
# this is our converted text to numerical representation from the Count vectorizer
cv_arr

<2419x65 sparse matrix of type '<class 'numpy.int64'>'
	with 45334 stored elements in Compressed Sparse Row format>

In [53]:
# Creating vocabulary array which will represent all the corpus 
vocab_tf_idf = tf_idf_vectorizer.get_feature_names()

# get the vocb list
vocab_tf_idf

C:\Users\Ankita\.conda\envs\abc\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\u200b',
 '’',
 '“',
 '”']

In [54]:
# Creating vocabulary array which will represent all the corpus 
vocab_cv = cv_vectorizer.get_feature_names()

# get the vocb list
vocab_cv

[' ',
 '%',
 '&',
 '*',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '°',
 '½',
 'ß',
 'è',
 'é',
 'ö',
 '˜',
 '\u200b',
 '\u200d',
 '\u200e',
 '–',
 '—',
 '’',
 '“',
 '”',
 '•',
 '…',
 '€',
 '→',
 '►',
 '★',
 '☆',
 '☝',
 '☹',
 '☺',
 '♀',
 '♂',
 '♡',
 '✅',
 '✨',
 '❤',
 '⭐',
 '️']

In [55]:
display(len(vocab_tf_idf))
display(len(vocab_cv))

31

65

# Implementing LDA

In [56]:
# Importing Gensim
import gensim
from gensim import corpora
from gensim.models import Phrases

In [57]:
df

date        asin  year  weeknum  fiscal_year  \
date_asin                                                                  
2020-12-20B07ZPDV5NJ  2020-12-20  B07ZPDV5NJ  2020       51         2020   
2020-12-20B087ZG9VTV  2020-12-20  B087ZG9VTV  2020       51         2020   
2020-12-21B07BVW522M  2020-12-21  B07BVW522M  2020       51         2020   
2020-12-21B087ZFLY3K  2020-12-21  B087ZFLY3K  2020       51         2020   
2020-12-21B087ZG94GD  2020-12-21  B087ZG94GD  2020       51         2020   
...                          ...         ...   ...      ...          ...   
2022-06-22B087ZG94GD  2022-06-22  B087ZG94GD  2022       25         2022   
2022-06-22B09GYRXP9P  2022-06-22  B09GYRXP9P  2022       25         2022   
2022-06-22B09NLL2YSF  2022-06-22  B09NLL2YSF  2022       25         2022   
2022-06-23B082VP1JR6  2022-06-23  B082VP1JR6  2022       25         2022   
2022-06-23B082VP1JR6  2022-06-23  B082VP1JR6  2022       25         2022   

                      fiscal_month  first_party_ordered_sales_vc  \
date_asin                                                          
2020-12-20B07ZPDV5NJ            12                          89.1   
2020-12-20B087ZG9VTV            12                         304.0   
2020-12-21B07BVW522M            12                         151.9   
2020-12-21B087ZFLY3K            12                         105.0   
2020-12-21B087ZG94GD            12                         289.1   
...                            ...                           ...   
2022-06-22B087ZG94GD             6                          56.8   
2022-06-22B09GYRXP9P             6                          60.1   
2022-06-22B09NLL2YSF             6                          84.5   
2022-06-23B082VP1JR6             6                         176.0   
2022-06-23B082VP1JR6             6                         176.0   

                      first_party_ordered_sales_local_vc  \
date_asin                                                  
2020-12-20B07ZPDV5NJ                               78.02   
2020-12-20B087ZG9VTV                              266.24   
2020-12-21B07BVW522M                              133.03   
2020-12-21B087ZFLY3K                               91.92   
2020-12-21B087ZG94GD                              253.21   
...                                                  ...   
2022-06-22B087ZG94GD                               47.38   
2022-06-22B09GYRXP9P                               50.10   
2022-06-22B09NLL2YSF                               70.50   
2022-06-23B082VP1JR6                              146.81   
2022-06-23B082VP1JR6                              146.81   

                      first_party_ordered_units_vc  average_sales_price_vc  \
date_asin                                                                    
2020-12-20B07ZPDV5NJ                            22                     3.5   
2020-12-20B087ZG9VTV                            64                     4.2   
2020-12-21B07BVW522M                            17                     7.8   
2020-12-21B087ZFLY3K                            26                     3.5   
2020-12-21B087ZG94GD                            71                     3.6   
...                                            ...                     ...   
2022-06-22B087ZG94GD                            11                     4.3   
2022-06-22B09GYRXP9P                             6                     8.3   
2022-06-22B09NLL2YSF                             3                    23.5   
2022-06-23B082VP1JR6                            41                     3.6   
2022-06-23B082VP1JR6                            41                     3.6   

                      ...  useful_ratings_negative  useful_ratings_positive  \
date_asin             ...                                                     
2020-12-20B07ZPDV5NJ  ...                        1                        3   
2020-12-20B087ZG9VTV  ...                        0                        1   
2020-12-21B07BVW522M  ...                 

# Implemented LDA on all 3 sub-categories with the same code

Taking cleansers here as an example

In [58]:
X=df_cleansers_pos.text_pr_translated
Y=df_cleansers_pos.conversion_rate_vc

In [59]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=123)

In [60]:
train_X

date_asin
2021-02-03B082VPVQZS    gel great removing makeup need small amounts g...
2020-12-31B08HT315T6    actually really ticklish skin likes irritate h...
2021-05-13B082VPVQZS                                         love product
2022-06-12B09NLL2YSF    neutrogena gift set consists products cleansin...
2022-06-13B09NLL2YSF    neutrogena hydro boost gift set packed neutral...
                                              ...                        
2021-05-07B08HT315T6                  fast delivery everything fine thank
2021-01-29B071936B11                                beautiful buy anytime
2022-03-15B082VPVQZS    use cleaning gel every day suitable sensitive ...
2021-05-27B08HT315T6                                        great product
2022-06-09B09NS8Q51H    son severe acne years cant list care products ...
Name: text_pr_translated, Length: 240, dtype: object

# For Positive Sentiments

In [61]:
X_train_tokens = [doc.split() for doc in train_X]  

In [84]:
X_train_tokens

[['gel',
  'great',
  'removing',
  'makeup',
  'need',
  'small',
  'amounts',
  'great',
  'sensitive',
  'skin'],
 ['actually',
  'really',
  'ticklish',
  'skin',
  'likes',
  'irritate',
  'however',
  'neutrogena',
  'company',
  'skin',
  'fewest',
  'problems',
  'bucks',
  'around',
  'hydro',
  'boost',
  'set',
  'covers',
  'cleaning',
  'maintenance',
  'included',
  'aqua',
  'cleansing',
  'gel',
  'aqua',
  'pearls',
  'serum',
  'aqua',
  'gel',
  'think',
  'good',
  'get',
  'long',
  'way',
  'products',
  'even',
  'small',
  'amount',
  'skin',
  'feels',
  'good',
  'products',
  'notice',
  'skin',
  'less',
  'greasy',
  'less',
  'greasy',
  'im',
  'sure',
  'serum',
  'cleansing',
  'gel',
  'already',
  'favorite',
  'products',
  'well',
  'imagine',
  'buying',
  'aqua',
  'gel',
  'later'],
 ['love', 'product'],
 ['neutrogena',
  'gift',
  'set',
  'consists',
  'products',
  'cleansing',
  'gel',
  'serum',
  'day',
  'cream',
  'packaging',
  'doesnt',

In [62]:
dictionary = corpora.Dictionary(X_train_tokens)
print(dictionary)

Dictionary(1332 unique tokens: ['amounts', 'gel', 'great', 'makeup', 'need']...)


In [63]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in X_train_tokens]

In [64]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [65]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary)

In [66]:
print(ldamodel.print_topics(num_topics=5, num_words=30))

[(0, '0.026*"product" + 0.026*"good" + 0.018*"skin" + 0.017*"gel" + 0.011*"great" + 0.011*"also" + 0.010*"use" + 0.009*"face" + 0.008*"neutrogena" + 0.006*"well" + 0.006*"little" + 0.006*"really" + 0.006*"clean" + 0.006*"price" + 0.006*"cleans" + 0.006*"far" + 0.006*"day" + 0.005*"pump" + 0.005*"time" + 0.005*"love" + 0.005*"used" + 0.005*"fresh" + 0.005*"washing" + 0.005*"clear" + 0.005*"give" + 0.004*"visible" + 0.004*"pimples" + 0.004*"sensitive" + 0.004*"already" + 0.004*"satisfied"'), (1, '0.026*"skin" + 0.019*"product" + 0.016*"great" + 0.015*"really" + 0.013*"gel" + 0.010*"clean" + 0.010*"doesnt" + 0.008*"buy" + 0.008*"use" + 0.008*"dry" + 0.007*"face" + 0.007*"afterwards" + 0.007*"water" + 0.007*"super" + 0.007*"also" + 0.007*"like" + 0.007*"well" + 0.007*"pimples" + 0.005*"better" + 0.005*"mask" + 0.005*"works" + 0.005*"feels" + 0.005*"using" + 0.005*"since" + 0.005*"washing" + 0.005*"serum" + 0.005*"soft" + 0.005*"feel" + 0.005*"used" + 0.004*"micellar"'), (2, '0.051*"skin" +

In [67]:
topics = ldamodel.show_topics(formatted=False, num_words=30)
    
for t in range(len(topics)):
    print("\nTopic {}, top {} words:".format(t+1,30))
    print(" ".join([w[0] for w in topics[t][1]]))


Topic 1, top 30 words:
product good skin gel great also use face neutrogena well little really clean price cleans far day pump time love used fresh washing clear give visible pimples sensitive already satisfied

Topic 2, top 30 words:
skin product great really gel clean doesnt buy use dry face afterwards water super also like well pimples better mask works feels using since washing serum soft feel used micellar

Topic 3, top 30 words:
skin gel good use cleansing products well cleaning pleasant also feels face smell product washing great aqua really afterwards much dont neutrogena sensitive dry recommend serum makeup even time super

Topic 4, top 30 words:
skin gel cleansing products use set serum also good aqua like cleaning ive really sensitive neutrogena feels great care pleasant recommend face makeup gift much feeling facial boost product cardboard

Topic 5, top 30 words:
skin gel face great care good neutrogena product water used fresh feel products one satisfied makeup cream also

In [68]:
ldamodel.print_topics(num_topics=5, num_words=30)

[(0,
  '0.026*"product" + 0.026*"good" + 0.018*"skin" + 0.017*"gel" + 0.011*"great" + 0.011*"also" + 0.010*"use" + 0.009*"face" + 0.008*"neutrogena" + 0.006*"well" + 0.006*"little" + 0.006*"really" + 0.006*"clean" + 0.006*"price" + 0.006*"cleans" + 0.006*"far" + 0.006*"day" + 0.005*"pump" + 0.005*"time" + 0.005*"love" + 0.005*"used" + 0.005*"fresh" + 0.005*"washing" + 0.005*"clear" + 0.005*"give" + 0.004*"visible" + 0.004*"pimples" + 0.004*"sensitive" + 0.004*"already" + 0.004*"satisfied"'),
 (1,
  '0.026*"skin" + 0.019*"product" + 0.016*"great" + 0.015*"really" + 0.013*"gel" + 0.010*"clean" + 0.010*"doesnt" + 0.008*"buy" + 0.008*"use" + 0.008*"dry" + 0.007*"face" + 0.007*"afterwards" + 0.007*"water" + 0.007*"super" + 0.007*"also" + 0.007*"like" + 0.007*"well" + 0.007*"pimples" + 0.005*"better" + 0.005*"mask" + 0.005*"works" + 0.005*"feels" + 0.005*"using" + 0.005*"since" + 0.005*"washing" + 0.005*"serum" + 0.005*"soft" + 0.005*"feel" + 0.005*"used" + 0.004*"micellar"'),
 (2,
  '0.051*

In [69]:
# printing the topic associaltions with the documents
count = 0
for i in ldamodel[doc_term_matrix]:
    print("doc : ",count,i)
    count += 1

doc :  0 [(0, 0.018453881), (1, 0.018503593), (2, 0.018715177), (3, 0.018552788), (4, 0.9257746)]
doc :  1 [(2, 0.98710334)]
doc :  2 [(0, 0.7301413), (1, 0.068031184), (2, 0.06701227), (3, 0.067381926), (4, 0.06743338)]
doc :  3 [(3, 0.6907402), (4, 0.30087858)]
doc :  4 [(2, 0.06547571), (3, 0.92913437)]
doc :  5 [(0, 0.1029207), (1, 0.10212906), (2, 0.10114509), (3, 0.10212607), (4, 0.5916791)]
doc :  6 [(0, 0.06888083), (1, 0.7288442), (2, 0.06786534), (3, 0.0672803), (4, 0.06712935)]
doc :  7 [(0, 0.729919), (1, 0.06728001), (2, 0.06725844), (3, 0.068525344), (4, 0.06701719)]
doc :  8 [(0, 0.018320067), (1, 0.92662734), (2, 0.018374883), (3, 0.018346133), (4, 0.018331518)]
doc :  9 [(0, 0.010747211), (1, 0.010673613), (2, 0.957148), (3, 0.010679647), (4, 0.010751524)]
doc :  10 [(0, 0.015705274), (1, 0.015721634), (2, 0.5370443), (3, 0.41592696), (4, 0.015601806)]
doc :  11 [(2, 0.9916297)]
doc :  12 [(0, 0.9095652), (1, 0.022806477), (2, 0.022514958), (3, 0.022517918), (4, 0.0225

In [77]:
def format_topics_sentences(ldamodel=ldamodel, corpus=doc_term_matrix, texts=X_train_tokens):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [82]:
import pandas as pd

df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=doc_term_matrix, texts=X_train_tokens)

C:\Users\Ankita\AppData\Local\Temp\ipykernel_21520\3961586387.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
C:\Users\Ankita\AppData\Local\Temp\ipykernel_21520\3961586387.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [83]:
df_topic_sents_keywords

Dominant_Topic  Perc_Contribution  \
0                 4             0.9258   
1                 2             0.9871   
2                 0             0.7301   
3                 3             0.6907   
4                 3             0.9320   
..              ...                ...   
235               3             0.8649   
236               1             0.7992   
237               3             0.9374   
238               0             0.7275   
239               1             0.9874   

                                        Topic_Keywords  \
0    skin, gel, face, great, care, good, neutrogena...   
1    skin, gel, good, use, cleansing, products, wel...   
2    product, good, skin, gel, great, also, use, fa...   
3    skin, gel, cleansing, products, use, set, seru...   
4    skin, gel, cleansing, products, use, set, seru...   
..                                                 ...   
235  skin, gel, cleansing, products, use, set, seru...   
236  skin, product, great, really, gel, clean, does...   
237  skin, gel, cleansing, products, use, set, seru...   
238  product, good, skin, gel, great, also, use, fa...   
239  skin, product, great, really, gel, clean, does...   

                                                     0  
0    [gel, great, removing, makeup, need, small, am...  
1    [actually, really, ticklish, skin, likes, irri...  
2                                      [love, product]  
3    [neutrogena, gift, set, consists, products, cl...  
4    [neutrogena, hydro, boost, gift, set, packed, ...  
..                                                 ...  
235          [fast, delivery, everything, fine, thank]  
236                          [beautiful, buy, anytime]  
237  [use, cleaning, gel, every, day, suitable, sen...  
238                                   [great, product]  
239  [son, severe, acne, years, cant, list, care, p...  

[240 rows x 4 columns]

In [86]:
# Format
df_dominant_topic_pos = df_topic_sents_keywords.reset_index()
df_dominant_topic_pos.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic_pos.head(10)

Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0               4              0.9258   
1            1               2              0.9871   
2            2               0              0.7301   
3            3               3              0.6907   
4            4               3              0.9320   
5            5               4              0.5917   
6            6               1              0.7288   
7            7               0              0.7299   
8            8               1              0.9266   
9            9               2              0.9571   

                                            Keywords  \
0  skin, gel, face, great, care, good, neutrogena...   
1  skin, gel, good, use, cleansing, products, wel...   
2  product, good, skin, gel, great, also, use, fa...   
3  skin, gel, cleansing, products, use, set, seru...   
4  skin, gel, cleansing, products, use, set, seru...   
5  skin, gel, face, great, care, good, neutrogena...   
6  skin, product, great, really, gel, clean, does...   
7  product, good, skin, gel, great, also, use, fa...   
8  skin, product, great, really, gel, clean, does...   
9  skin, gel, good, use, cleansing, products, wel...   

                                                Text  
0  [gel, great, removing, makeup, need, small, am...  
1  [actually, really, ticklish, skin, likes, irri...  
2                                    [love, product]  
3  [neutrogena, gift, set, consists, products, cl...  
4  [neutrogena, hydro, boost, gift, set, packed, ...  
5                                        [satisfied]  
6                                   [super, product]  
7                                    [good, quality]  
8  [doesnt, clean, well, panda, eyes, nothing, cl...  
9  [dry, sensitive, skin, products, used, togethe...

In [128]:
df_dominant_topic_pos['Keywords'].nunique()

5

In [129]:
df_dominant_topic_pos.to_csv('C:/Users/Ankita/OneDrive - Infiniti Research/Desktop/Common_PositiveTopics_Cleansers.csv')

In [46]:
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", ldamodel.log_perplexity(doc_term_matrix))

Perplexity:  -7.042290909531466


# For Negative Sentiments

In [130]:
X1=df_cleansers_neg.text_pr_translated
Y1=df_cleansers_neg.conversion_rate_vc

In [131]:
train_X, test_X, train_y, test_y = train_test_split(X1, Y1, test_size=0.3, random_state=123)

In [132]:
train_X

837     quality match description removes waterproof m...
2248    use night cleanse dry skin longer take water t...
2224    ive already tried several peelings rosacea wor...
1561      got package like says description way expensive
1962    struggle dry skin especially since winter say ...
758     bought product prime day old cream running ski...
2005           bad burns bit op thats normal dont know‍♀️
2100    nothing people sensitive smells expensive pric...
2429    serum smells disgusting rude sell something li...
1401    pack arrives damaged used anyway disappointed ...
2147    smell really disgustingam otherwise picky sens...
1026    unfortunately seem made total mistake facial t...
1402    pack arrives damaged used anyway disappointed ...
2420    using keep mind dont daily surface cleaner pre...
308     product smells bit artificial looks rather fak...
309     product smells bit artificial looks rather fak...
2074    face always burns slightly use guess thats sup...
2111    never 

In [133]:
X_train_tokens = [doc.split() for doc in train_X]  

In [134]:
dictionary = corpora.Dictionary(X_train_tokens)
print(dictionary)

Dictionary(293 unique tokens: ['artificial', 'description', 'makeup', 'match', 'quality']...)


In [135]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in X_train_tokens]

In [136]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [137]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary)

In [138]:
print(ldamodel.print_topics(num_topics=5, num_words=20))

[(0, '0.023*"product" + 0.019*"skin" + 0.016*"dry" + 0.014*"description" + 0.014*"products" + 0.013*"face" + 0.013*"acne" + 0.011*"like" + 0.011*"especially" + 0.010*"got" + 0.010*"recommended" + 0.010*"artificial" + 0.009*"pimples" + 0.009*"day" + 0.009*"smells" + 0.008*"works" + 0.008*"back" + 0.008*"waterproof" + 0.008*"without" + 0.008*"cream"'), (1, '0.024*"skin" + 0.022*"unfortunately" + 0.016*"also" + 0.014*"days" + 0.014*"product" + 0.012*"use" + 0.012*"bad" + 0.011*"day" + 0.011*"pimples" + 0.011*"thats" + 0.010*"dont" + 0.010*"face" + 0.010*"well" + 0.009*"first" + 0.009*"used" + 0.009*"like" + 0.008*"got" + 0.008*"works" + 0.008*"normal" + 0.007*"rub"'), (2, '0.016*"serum" + 0.016*"like" + 0.015*"something" + 0.015*"smells" + 0.015*"disgusting" + 0.015*"sell" + 0.015*"rude" + 0.015*"ton" + 0.015*"€" + 0.009*"areas" + 0.007*"works" + 0.007*"mind" + 0.007*"ive" + 0.007*"pimples" + 0.007*"disappear" + 0.007*"spared" + 0.007*"problem" + 0.007*"treated" + 0.007*"surface" + 0.006*

In [139]:
topics = ldamodel.show_topics(formatted=False, num_words=30)
    
for t in range(len(topics)):
    print("\nTopic {}, top {} words:".format(t+1,30))
    print(" ".join([w[0] for w in topics[t][1]]))


Topic 1, top 30 words:
product skin dry description products face acne like especially got recommended artificial pimples day smells works back waterproof without cream makeup every expensive small winter since say says feeling way

Topic 2, top 30 words:
skin unfortunately also days product use bad day pimples thats dont face well first used like got works normal rub rash burns facial hard bought waterproof description application extremely ive

Topic 3, top 30 words:
serum like something smells disgusting sell rude ton € areas works mind ive pimples disappear spared problem treated surface cleaner using redness daily weeks days subsides far trying dont keep

Topic 4, top 30 words:
used skin works ive im without irritating cools even rosacea excited already tried peelings several dont think anyway pack afterwards arrives damaged ratio fair disappointed priceperformance product like neutrogena gel

Topic 5, top 30 words:
skin product smells buy also facial dry neutrogena products bit 

In [140]:
ldamodel.print_topics(num_topics=5, num_words=30)

[(0,
  '0.023*"product" + 0.019*"skin" + 0.016*"dry" + 0.014*"description" + 0.014*"products" + 0.013*"face" + 0.013*"acne" + 0.011*"like" + 0.011*"especially" + 0.010*"got" + 0.010*"recommended" + 0.010*"artificial" + 0.009*"pimples" + 0.009*"day" + 0.009*"smells" + 0.008*"works" + 0.008*"back" + 0.008*"waterproof" + 0.008*"without" + 0.008*"cream" + 0.007*"makeup" + 0.007*"every" + 0.007*"expensive" + 0.007*"small" + 0.007*"winter" + 0.007*"since" + 0.007*"say" + 0.007*"says" + 0.007*"feeling" + 0.007*"way"'),
 (1,
  '0.024*"skin" + 0.022*"unfortunately" + 0.016*"also" + 0.014*"days" + 0.014*"product" + 0.012*"use" + 0.012*"bad" + 0.011*"day" + 0.011*"pimples" + 0.011*"thats" + 0.010*"dont" + 0.010*"face" + 0.010*"well" + 0.009*"first" + 0.009*"used" + 0.009*"like" + 0.008*"got" + 0.008*"works" + 0.008*"normal" + 0.007*"rub" + 0.007*"rash" + 0.007*"burns" + 0.007*"facial" + 0.007*"hard" + 0.007*"bought" + 0.007*"waterproof" + 0.007*"description" + 0.007*"application" + 0.006*"extreme

In [141]:
# printing the topic associaltions with the documents
count = 0
for i in ldamodel[doc_term_matrix]:
    print("doc : ",count,i)
    count += 1

doc :  0 [(0, 0.93810266), (1, 0.015478779), (2, 0.015441504), (3, 0.015441848), (4, 0.015535237)]
doc :  1 [(0, 0.020183189), (1, 0.020188682), (2, 0.020010388), (3, 0.020033235), (4, 0.9195846)]
doc :  2 [(0, 0.013424684), (1, 0.013459528), (2, 0.013354557), (3, 0.94633234), (4, 0.0134289)]
doc :  3 [(0, 0.899161), (1, 0.025510298), (2, 0.025119578), (3, 0.025041187), (4, 0.025167974)]
doc :  4 [(0, 0.9463491), (1, 0.01345779), (2, 0.013351682), (3, 0.01339162), (4, 0.013449817)]
doc :  5 [(0, 0.98378164)]
doc :  6 [(0, 0.022329975), (1, 0.9106233), (2, 0.022244956), (3, 0.022299163), (4, 0.022502575)]
doc :  7 [(0, 0.018321697), (1, 0.018263241), (2, 0.01821374), (3, 0.018209022), (4, 0.9269923)]
doc :  8 [(0, 0.020094771), (1, 0.02007638), (2, 0.919643), (3, 0.0200245), (4, 0.020161392)]
doc :  9 [(0, 0.015398043), (1, 0.015602571), (2, 0.01539601), (3, 0.9382087), (4, 0.015394653)]
doc :  10 [(0, 0.9698994)]
doc :  11 [(4, 0.973105)]
doc :  12 [(0, 0.015398043), (1, 0.015604273), 

In [142]:
def format_topics_sentences(ldamodel=ldamodel, corpus=doc_term_matrix, texts=X_train_tokens):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [143]:
import pandas as pd

df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=doc_term_matrix, texts=X_train_tokens)

C:\Users\Ankita\AppData\Local\Temp\ipykernel_18396\3961586387.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
C:\Users\Ankita\AppData\Local\Temp\ipykernel_18396\3961586387.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [144]:
# Format
df_dominant_topic_neg = df_topic_sents_keywords.reset_index()
df_dominant_topic_neg.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic_neg.head(10)

Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0               0              0.9381   
1            1               4              0.9196   
2            2               3              0.9463   
3            3               0              0.8991   
4            4               0              0.9463   
5            5               0              0.9838   
6            6               1              0.9106   
7            7               4              0.9270   
8            8               2              0.9196   
9            9               3              0.9382   

                                            Keywords  \
0  product, skin, dry, description, products, fac...   
1  skin, product, smells, buy, also, facial, dry,...   
2  used, skin, works, ive, im, without, irritatin...   
3  product, skin, dry, description, products, fac...   
4  product, skin, dry, description, products, fac...   
5  product, skin, dry, description, products, fac...   
6  skin, unfortunately, also, days, product, use,...   
7  skin, product, smells, buy, also, facial, dry,...   
8  serum, like, something, smells, disgusting, se...   
9  used, skin, works, ive, im, without, irritatin...   

                                                Text  
0  [quality, match, description, removes, waterpr...  
1  [use, night, cleanse, dry, skin, longer, take,...  
2  [ive, already, tried, several, peelings, rosac...  
3  [got, package, like, says, description, way, e...  
4  [struggle, dry, skin, especially, since, winte...  
5  [bought, product, prime, day, old, cream, runn...  
6  [bad, burns, bit, op, thats, normal, dont, kno...  
7  [nothing, people, sensitive, smells, expensive...  
8  [serum, smells, disgusting, rude, sell, someth...  
9  [pack, arrives, damaged, used, anyway, disappo...

In [145]:
df_dominant_topic_neg.Dominant_Topic.nunique()


5

In [146]:
df_dominant_topic_neg.to_csv('C:/Users/Ankita/OneDrive - Infiniti Research/Desktop/Common_Negative_TopicsCleansers.csv')

In [149]:
trans_data=pd.read_excel(r'C:/Users/Ankita/OneDrive - Infiniti Research/Desktop/Translated Mice Data.xlsx')

In [161]:
from datetime import timedelta, datetime, date

In [163]:
trans_data.date=pd.to_datetime(trans_data.date,format='%Y-%m-%d')

# Merging all the Topics that we got in all the 3 sub categories (after manual inspection and validation in excel) with the final dataset

In [153]:
Topic_data=pd.read_excel(r'C:/Users/Ankita/OneDrive - Infiniti Research/Desktop/FinalData_withSC&Topics.xlsx')

In [156]:
Topic_data.dropna(subset=['text_pr_translated'],inplace=True)

In [165]:
Topic_data=Topic_data[['date','asin','Top Keywords','Topic']]

In [168]:
Topic_data.date=pd.to_datetime(Topic_data.date,format='%Y-%m-%d')

C:\Users\Ankita\AppData\Local\Temp\ipykernel_18396\865705783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Topic_data.date=pd.to_datetime(Topic_data.date,format='%Y-%m-%d')


In [169]:
Topic_data

date        asin                                   Top Keywords  \
5     2020-12-20  B07ZPDV5NJ                                     burns skin   
20    2020-12-20  B087ZG9VTV                             effective dry skin   
26    2020-12-21  B07BVW522M                                            yes   
41    2020-12-21  B087ZFLY3K                                     burns skin   
42    2020-12-21  B087ZG94GD                                     burns skin   
...          ...         ...                                            ...   
14318 2022-06-22  B087ZFLY3K                             highly recommended   
14319 2022-06-22  B087ZG94GD                             highly recommended   
14324 2022-06-22  B09GYRXP9P                   practical packaging product    
14326 2022-06-22  B09NLL2YSF  recommend 2 out of 3 products in the gift box   
14339 2022-06-23  B082VP1JR6                              delivery disaster   

                              Topic  
5                  Product Feedback  
20                 Product Feedback  
26                 Product Feedback  
41                 Product Feedback  
42                 Product Feedback  
...                             ...  
14318              Product Feedback  
14319              Product Feedback  
14324  Product Feedback & Packaging  
14326              Product Feedback  
14339                      Delivery  

[2029 rows x 4 columns]

In [171]:
trans_data.shape

(2437, 130)

In [172]:
final_df = trans_data.merge(Topic_data,on=['date','asin'],how = 'left',)

In [173]:
final_df

date        asin  year  weeknum  fiscal_year  fiscal_month  \
0    2020-12-20  B07ZPDV5NJ  2020       51         2020            12   
1    2020-12-20  B087ZG9VTV  2020       51         2020            12   
2    2020-12-21  B07BVW522M  2020       51         2020            12   
3    2020-12-21  B087ZFLY3K  2020       51         2020            12   
4    2020-12-21  B087ZFLY3K  2020       51         2020            12   
...         ...         ...   ...      ...          ...           ...   
2432 2022-06-22  B087ZG94GD  2022       25         2022             6   
2433 2022-06-22  B09GYRXP9P  2022       25         2022             6   
2434 2022-06-22  B09NLL2YSF  2022       25         2022             6   
2435 2022-06-23  B082VP1JR6  2022       25         2022             6   
2436 2022-06-23  B082VP1JR6  2022       25         2022             6   

      first_party_ordered_sales_vc  first_party_ordered_sales_local_vc  \
0                             89.1                               78.02   
1                            304.0                              266.24   
2                            151.9                              133.03   
3                            105.0                               91.92   
4                            105.0                               91.92   
...                            ...                                 ...   
2432                          56.8                               47.38   
2433                          60.1                               50.10   
2434                          84.5                               70.50   
2435                         176.0                              146.81   
2436                         176.0                              146.81   

      first_party_ordered_units_vc  average_sales_price_vc  ...  \
0                               22                     3.5  ...   
1                               64                     4.2  ...   
2                               17                     7.8  ...   
3                               26                     3.5  ...   
4                               26                     3.5  ...   
...                            ...                     ...  ...   
2432                            11                     4.3  ...   
2433                             6                     8.3  ...   
2434                             3                    23.5  ...   
2435                            41                     3.6  ...   
2436                            41                     3.6  ...   

      useful_ratings_positive  feature_wise_rating  qna_answered  \
0                           3                    1             4   
1                           1                    1             4   
2                           4                    1             6   
3                           3                    1             4   
4                           3                    1             4   
...                       ...                  ...           ...   
2432                        3                    1             4   
2433                        1                    1             0   
2434                        1                    0             0   
2435                        1                    1             5   
2436                        1                    1             5   

                                                  title  \
0     Neutrogena Anti-Pickel Gesichtspflege, SOS Sof...   
1     Neutrogena Curcuma Clear Beruhigender Reinigun...   
2     Neutrogena Hydro Boost Hyaluron Serum, Aqua Pe...   
3     Neutrogena Curcuma Clear Gesichtsreinigung, Be...   
4     Neutrogena Curcuma Clear Gesichtsreinigung, Be...   
...                                                 ...   
2432  Neutrogena Curcuma Clear Beruhigende Feuchtigk...   
2433  Neutrogena Hydro Boost Hyaluron Konzentrat (15...   
2434  Neutrogena Hydro Boost Geschenk Set Gesichtspf...   
2435  Neutrogena Anti-Mitesser Ge

In [174]:
final_df.to_csv('C:/Users/Ankita/OneDrive - Infiniti Research/Desktop/TopicTranslatedData.csv')

In [175]:
final_df.head()

date        asin  year  weeknum  fiscal_year  fiscal_month  \
0 2020-12-20  B07ZPDV5NJ  2020       51         2020            12   
1 2020-12-20  B087ZG9VTV  2020       51         2020            12   
2 2020-12-21  B07BVW522M  2020       51         2020            12   
3 2020-12-21  B087ZFLY3K  2020       51         2020            12   
4 2020-12-21  B087ZFLY3K  2020       51         2020            12   

   first_party_ordered_sales_vc  first_party_ordered_sales_local_vc  \
0                          89.1                               78.02   
1                         304.0                              266.24   
2                         151.9                              133.03   
3                         105.0                               91.92   
4                         105.0                               91.92   

   first_party_ordered_units_vc  average_sales_price_vc  ...  \
0                            22                     3.5  ...   
1                            64                     4.2  ...   
2                            17                     7.8  ...   
3                            26                     3.5  ...   
4                            26                     3.5  ...   

   useful_ratings_positive  feature_wise_rating  qna_answered  \
0                        3                    1             4   
1                        1                    1             4   
2                        4                    1             6   
3                        3                    1             4   
4                        3                    1             4   

                                               title  \
0  Neutrogena Anti-Pickel Gesichtspflege, SOS Sof...   
1  Neutrogena Curcuma Clear Beruhigender Reinigun...   
2  Neutrogena Hydro Boost Hyaluron Serum, Aqua Pe...   
3  Neutrogena Curcuma Clear Gesichtsreinigung, Be...   
4  Neutrogena Curcuma Clear Gesichtsreinigung, Be...   

                                             text_pr  \
0  Bei mir hilft es leider nicht. Beim Auftragen ...   
1  Wirkt und trocknet die Haut nicht aus. Habe sc...   
2                                                 Ja   
3  Brennt auf der Haut, danach ist die Haut schli...   
4                                                ☹☹☹   

                                  text_pr_translated  \
0  Unfortunately it doesn't help me. A burning se...   
1  Effective and does not dry out the skin. I've ...   
2                                                And   
3  Burns on the skin, then the skin is worse than...   
4                                                NaN   

                                    translated_title  text_sentiment_sc  \
0  Neutrogena anti-pimple face care, SOS immediat...           0.040000   
1  Neutrogena Curcuma Clear Soothing cleansing fo...           0.383333   
2  Neutrogena Hydro Boost Hyaluron Serum, aqua pe...           0.000000   
3  Neutrogena Curcuma Clear facial cleansing, soo...          -0.122222   
4  Neutrogena Curcuma Clear facial cleansing, soo...           0.000000   

         Top Keywords             Topic  
0          burns skin  Product Feedback  
1  effective dry skin  Product Feedback  
2                 yes  Product Feedback  
3          burns skin  Product Feedback  
4          burns skin  Product Feedback  

[5 rows x 132 columns]